In [1]:
import pandas as pd
import numpy as np
import gensim
import random
import sys
import glob
import os
import datetime
from nltk import sent_tokenize
from nltk import word_tokenize
from scipy.spatial.distance import cosine
import warnings
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string
from gensim.utils import simple_preprocess
from gensim.models.callbacks import CallbackAny2Vec
from itertools import product
from sklearn.metrics import precision_recall_curve

warnings.simplefilter('ignore')

sys.path.append("../../oats")
from oats.annotation.ontology import Ontology
from oats.distances import pairwise as pw
from oats.utils.utils import flatten

In [2]:
1-cosine([0.23,0.51,0.72,0.61,0,0,0], [0,0,0,0.61,0.34,0.56,0.79])
1-cosine([0.23,0.51,0.72,0.61,0.12,0.15,0.18], [0.22,0.14,0.12,0.61,0.34,0.56,0.79])
1-cosine([0.23,0.53,0.72,0.61], [0.23,0.53,0.72,0.61])

1.0

In [3]:
class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epochs = []
        self.epoch = 1
        self.losses = []
        self.deltas = []
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 1:
            delta = loss
        else:
            delta = loss- self.loss_previous_step
        self.loss_previous_step=loss
        self.losses.append(loss)
        self.epochs.append(self.epoch)
        self.epoch += 1
        self.deltas.append(delta)

In [9]:
model = gensim.models.Word2Vec.load("../models/plants_sg/saved/word2vec_both_dim200_500_a_s_min3_window8.model")

In [5]:
w = preprocess_string("susceptible")[0]
model.most_similar(w,topn=5)

[('resist', 0.533036470413208),
 ('infect', 0.48954248428344727),
 ('pathogen', 0.4593661427497864),
 ('fungal', 0.4558146595954895),
 ('necrotroph', 0.42447996139526367)]

In [6]:
model_wiki = gensim.models.Word2Vec.load("../models/wiki_sg/word2vec.bin")

In [7]:
w = "susceptible"
model_wiki.most_similar(w,topn=6)

[('resistant', 0.7845903635025024),
 ('susceptibility', 0.7155444622039795),
 ('prone', 0.6828949451446533),
 ('vulnerable', 0.6462675929069519),
 ('affects', 0.6267737150192261),
 ('inhibit', 0.5860500335693359)]

In [18]:
# Trying to figure out if we can load weights directly using keyed vectors.
from gensim.models.keyedvectors import KeyedVectors

word_vectors = model.wv
word_vectors.save('vectors.kv')
reloaded_word_vectors = KeyedVectors.load('vectors.kv')



In [19]:
reloaded_word_vectors

In [36]:
kv = gensim.models.keyedvectors.Word2VecKeyedVectors(vector_size=5)
kv.add("something", np.array([0.2352,0.1212,0.2323,0.23423,0.2323]))
kv.add("else", np.array([0.9999,0.1212,0.2323,0.23423,0.2323]))
kv.add("another", np.array([0.2352,0.9999,0.2323,0.23423,0.2323]))
kv.save('vectors.kv')
kv

In [40]:
new_model = KeyedVectors.load("vectors.kv")
new_model["something"]

array([0.2352 , 0.1212 , 0.2323 , 0.23423, 0.2323 ])